In [9]:
import cv2 
import numpy as np
from tensorflow.keras.models import load_model
model = load_model('C:/Users/SAMEER/Desktop/jupytr_notebook_machine_learning/PROJECT_SURVEY CORPS/Mask_detection_real_time/weights.best.hdf5')

labels_dict = {0: 'without mask', 1:'mask'}
color_dict={0:(0,0,255), 1:(0,255,0)}

size = 4
#starting video capture
webcam = cv2.VideoCapture(0)
#loading the opencv classifier
classifier = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')


while True:
    (rval, im) = webcam.read()
    im = cv2.flip(im, 1, 1) #flip to act as a mirror
    
    # resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1]//size, im.shape[0]//size))
    
    # Detect multiscale/ face
    faces = classifier.detectMultiScale(mini)
    
    # draw rectangles around face
    for f in faces:
        (x, y, w, h) = [v * size for v in f]#Scale the shapesize backup
        #Sace just the rectangle face in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized = cv2.resize(face_img, (150, 150))
        normalized = resized/255.0
        reshaped = np.reshape(normalized,(1, 150, 150, 3))
        reshaped = np.vstack([reshaped])
        result = model.predict(reshaped)
        #print(result)
        
        label = np.argmax(result, axis=1)[0]
        
        cv2.rectangle(im, (x,y), (x+w, y+h), color_dict[label], 2)
        cv2.rectangle(im, (x,y-40), (x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x,y-10), cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    #show image
    cv2.imshow('LIVE', im)
    key = cv2.waitKey(10)
    #ESC key breaks the loop
    if key == 27:
        break
#shutting down webcam
webcam.release()
#clossing program window
cv2.destroyAllWindows()